In [1]:
from langchain import hub
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter, CharacterTextSplitter,TokenTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from langchain_openai import AzureChatOpenAI, AzureOpenAIEmbeddings, ChatOpenAI
import os
from pinecone import Pinecone, ServerlessSpec
from langchain_pinecone import PineconeVectorStore
from uuid import uuid4
from dotenv import load_dotenv
load_dotenv()

USER_AGENT environment variable not set, consider setting it to identify your requests.


True

In [2]:
AZURE_OPENAI_ENDPOINT = os.getenv('AZURE_OPENAI_ENDPOINT')
AZURE_OPENAI_DEPLOYMENT_ID = os.getenv('AZURE_OPENAI_DEPLOYMENT_ID')
AZURE_OPENAI_KEY = os.getenv('AZURE_OPENAI_KEY')
AZURE_API_VERSION = os.getenv('AZURE_API_VERSION')
PINECONE_API_KEY = os.getenv('PINECONE_API_KEY')
pc = Pinecone(api_key=PINECONE_API_KEY)

In [3]:
# llm = AzureChatOpenAI(
#             azure_endpoint=AZURE_OPENAI_ENDPOINT,
#             azure_deployment=AZURE_OPENAI_DEPLOYMENT_ID,
#             api_version=AZURE_API_VERSION,
#             api_key=AZURE_OPENAI_KEY,
#             temperature=0.0,
#             verbose=True,
#         )

# embedding_llm = AzureOpenAIEmbeddings(
#             azure_endpoint=AZURE_OPENAI_ENDPOINT,
#             azure_deployment='embedding-ada-crayon',
#             api_key=AZURE_OPENAI_KEY,
#             api_version=AZURE_API_VERSION,
#         )


llm = ChatOpenAI(
            temperature=0.0,
            verbose=True,
            model="gpt-4o-mini",
        )

embedding_llm = OpenAIEmbeddings(model="text-embedding-3-small")

# using code base setting for uploading to pinecone

In [4]:
import time

index_name = "ocbc-cv-gpt-3-small"  # change if desired

existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]

if index_name not in existing_indexes:
    pc.create_index(
        name=index_name,
        dimension=1536,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )
    while not pc.describe_index(index_name).status["ready"]:
        time.sleep(1)

index = pc.Index(index_name)

In [5]:
index_name = "ocbc-cv-gpt-3-small"  # change if desired
index = pc.Index(index_name)

In [10]:
# import os
# import comtypes.client

# def convert_doc_to_docx(doc_path):
#     # Initialize Word application
#     word = comtypes.client.CreateObject('Word.Application')
#     word.Visible = False

#     # Open the .doc file
#     doc = word.Documents.Open(doc_path)

#     # Get the .docx file path
#     docx_path = os.path.splitext(doc_path)[0] + ".docx"

#     # Save the file as .docx
#     doc.SaveAs(docx_path, FileFormat=16)  # 16 corresponds to the .docx format

#     # Close the document and quit Word
#     doc.Close()
#     word.Quit()

#     print(f"Converted '{doc_path}' to '{docx_path}'")

# if __name__ == "__main__":
#     # Specify the directory containing .doc files
#     directory = r"D:\OCBC_CRAYON_INTERNSHIP_DATA\OCBC_CV_RAG\job_word"

#     # Convert all .doc files in the directory to .docx
#     for filename in os.listdir(directory):
#         if filename.endswith(".doc"):
#             doc_path = os.path.join(directory, filename)
#             convert_doc_to_docx(doc_path)


KeyboardInterrupt: 

In [6]:
import os
from langchain_community.document_loaders import Docx2txtLoader, PyPDFLoader
from langchain.docstore.document import Document

def load_documents_from_folder(folder_path):
    documents = []

    # Iterate over files in the folder
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        
        # Check if the file is a .docx or .pdf and load accordingly
        if filename.endswith('.docx'):
            loader = Docx2txtLoader(file_path)
            documents.extend(loader.load())
        elif filename.endswith('.pdf'):
            loader = PyPDFLoader(file_path)
            documents.extend(loader.load())
    
    return documents

# Usage example
folder_path =  r"D:\OCBC_CRAYON_INTERNSHIP_DATA\OCBC_CV_RAG\job_word"
documents = load_documents_from_folder(folder_path)

# Now `documents` contains all loaded Document objects


In [7]:
documents

[Document(metadata={'source': 'D:\\OCBC_CRAYON_INTERNSHIP_DATA\\OCBC_CV_RAG\\job_word\\Acceptance Manager.docx'}, page_content='Job Description\n\n\n\n\tDATA JABATAN :\n\nJOB ID\n\nOPS037\n\nNAMA JABATAN\n\nAcceptance Manager \n\nGRUP\n\nOperations & IT \n\nREPORT TO POSITION\n\nLegal & Acceptance Head \n\nFUNGSI UTAMA\n\nBertanggung jawab untuk melakukan aktivitas fungsi dan koordinasi, aktivitas operasional Bank di area yang merupakan wilayah pengawasannya.\n\n\n\nTANGGUNG JAWAB/ TUGAS POKOK PEKERJAAN\n\nMengawasi dan memonitor transaksi pencairan kredit Retail Banking segment.\n\nMemastikan Fungsi Client Support sebagai counter trade dilaksanakan  sesuai dengan standar layanan (SLA) dan prosedur yang berlaku.\n\nMemonitor kinerja seluruh staff Client Support sebagai pendukung operasional Trade antara lain transaksi Ekspor, Impor, Bank Garansi dan Trade Loan,  dan dalam menangani proses pencairan, perubahan, angsuran dan pelunasan PIF/PEF by LC maupun non LC, TRF sehingga proses dapa

In [16]:
documents[0].metadata = {""}

{'source': 'D:\\OCBC_CRAYON_INTERNSHIP_DATA\\OCBC_CV_RAG\\job_word\\Acceptance Manager.docx'}

In [8]:
def modify_metadata(metadata):
    # Extract the 'source' value
    source_path = metadata.get('source', '')
    
    # Extract the last folder and file name without extension
    job_name = os.path.splitext(os.path.basename(source_path))[0]
    
    # Create new metadata with key 'Job' and the modified value
    new_metadata = {'Job': job_name}
    
    return new_metadata

In [9]:
from langchain_core.documents import Document

new_docs  = []

for old_doc in documents:
    new_metadata = modify_metadata(old_doc.metadata)
    new_docs.append(Document(page_content=old_doc.page_content, metadata=new_metadata))

In [10]:
new_docs

[Document(metadata={'Job': 'Acceptance Manager'}, page_content='Job Description\n\n\n\n\tDATA JABATAN :\n\nJOB ID\n\nOPS037\n\nNAMA JABATAN\n\nAcceptance Manager \n\nGRUP\n\nOperations & IT \n\nREPORT TO POSITION\n\nLegal & Acceptance Head \n\nFUNGSI UTAMA\n\nBertanggung jawab untuk melakukan aktivitas fungsi dan koordinasi, aktivitas operasional Bank di area yang merupakan wilayah pengawasannya.\n\n\n\nTANGGUNG JAWAB/ TUGAS POKOK PEKERJAAN\n\nMengawasi dan memonitor transaksi pencairan kredit Retail Banking segment.\n\nMemastikan Fungsi Client Support sebagai counter trade dilaksanakan  sesuai dengan standar layanan (SLA) dan prosedur yang berlaku.\n\nMemonitor kinerja seluruh staff Client Support sebagai pendukung operasional Trade antara lain transaksi Ekspor, Impor, Bank Garansi dan Trade Loan,  dan dalam menangani proses pencairan, perubahan, angsuran dan pelunasan PIF/PEF by LC maupun non LC, TRF sehingga proses dapat berjalan dengan baik dan optimal dan sesuai SLA.\n\nMemeriksa 

In [11]:
len(new_docs)

163

In [8]:
all_pages_final = []

for doc in all_pages:
    filename = doc.metadata.get('source').split('\\')[-1]  # Extract the filename
    doc.page_content = f"Filename: {filename}\n\n{doc.page_content}"  # Embed the filename
    
    all_pages_final.append(doc) 

In [8]:
pc = Pinecone(api_key=PINECONE_API_KEY)
text_splitter = TokenTextSplitter(chunk_size=500, chunk_overlap=200)
splits = text_splitter.split_documents(documents)

In [12]:
len(splits)

449

In [ ]:
pc = Pinecone(api_key=PINECONE_API_KEY)
vector_store = PineconeVectorStore(index=index, embedding=embedding_llm)
vector_store.add_documents(documents=splits)

In [12]:
pc = Pinecone(api_key=PINECONE_API_KEY)
vector_store = PineconeVectorStore(index=index, embedding=embedding_llm)

batch_size = 50
for i in range(0, len(new_docs), batch_size):
    batch = new_docs[i:i + batch_size]
    vector_store.add_documents(documents=batch)
    print("Successfully index", i, "to", i+batch_size)


Successfully index 0 to 50
Successfully index 50 to 100
Successfully index 100 to 150
Successfully index 150 to 200


In [25]:
retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={'k': 10})

In [26]:
retriever.invoke("cv")

[Document(metadata={'Job': 'JD ON_Software_Engineer 1'}, page_content='Job Description\n\n\n\n\n\n\tDATA JABATAN :\n\nJOB ID\n\nOPS540, OPS489, OPS536,OPS519,OPS526\n\nNAMA JABATAN\n\nSoftware Engineer\n\nGRUP\n\nOperations & Technology\n\nREPORT TO POSITION\n\nPlatform Lead Engineer\n\nFUNGSI UTAMA\n\nMenjalankan proses development, testing dan release sistem teknologi informasi sesuai dengan pedoman Agile Devops untuk memberikan value bisnis, serta memastikan kualitas deliverables secara konsisten dilaksanakan dengan penuh tanggungjawab\n\n\n\nTANGGUNG JAWAB/ TUGAS POKOK PEKERJAAN\n\n\tMemastikan penggunaan framework yang telah ditetapkan dalam pelaksanaan project dengan tetap mengacu pada Peraturan OJK sebagai landasan utama.\n\n\tMelaksanakan swakelola dan bekerja lintas fungsi dalam menyelesaikan pekerjaan sesuai backlog yang telah ditentukan.\n\n\tMenghasilkan produk bernilai lebih melalui proses development, testing, dan release yang memadai.\n\n\tBersama-sama membangun lingkara